# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 9:37AM,247239,10,PRF-37887,Bio-PRF,Released
1,Sep 26 2022 9:36AM,247238,16,AIC-379727,American International Chemical,Released
2,Sep 26 2022 9:28AM,247236,10,WT5352 Out,"Methapharm, Inc.",Released
3,Sep 26 2022 9:24AM,247235,10,Enova-10700,Emerginnova,Released
4,Sep 26 2022 9:22AM,247234,12,HH-37092,Hush Hush,Released
5,Sep 26 2022 9:22AM,247234,12,HH-37093,Hush Hush,Released
6,Sep 26 2022 9:22AM,247234,12,HH-37094,Hush Hush,Released
7,Sep 26 2022 9:22AM,247234,12,HH-37095,Hush Hush,Released
8,Sep 26 2022 9:22AM,247234,12,HH-37096,Hush Hush,Released
9,Sep 26 2022 9:22AM,247234,12,HH-37097,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,247234,Released,50
10,247235,Released,1
11,247236,Released,1
12,247238,Released,1
13,247239,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247234,NaN,50.0
247235,NaN,1.0
247236,NaN,1.0
247238,NaN,1.0
247239,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247215,0.0,1.0
247217,0.0,1.0
247219,0.0,15.0
247220,0.0,3.0
247222,0.0,28.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247215,0,1
247217,0,1
247219,0,15
247220,0,3
247222,0,28


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247215,0,1
1,247217,0,1
2,247219,0,15
3,247220,0,3
4,247222,0,28


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247215,,1
1,247217,,1
2,247219,,15
3,247220,,3
4,247222,,28


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 9:37AM,247239,10,Bio-PRF
1,Sep 26 2022 9:36AM,247238,16,American International Chemical
2,Sep 26 2022 9:28AM,247236,10,"Methapharm, Inc."
3,Sep 26 2022 9:24AM,247235,10,Emerginnova
4,Sep 26 2022 9:22AM,247234,12,Hush Hush
54,Sep 26 2022 9:20AM,247233,22,"NBTY Global, Inc."
55,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation
98,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation
158,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation
186,Sep 26 2022 8:46AM,247223,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 26 2022 9:37AM,247239,10,Bio-PRF,,1
1,Sep 26 2022 9:36AM,247238,16,American International Chemical,,1
2,Sep 26 2022 9:28AM,247236,10,"Methapharm, Inc.",,1
3,Sep 26 2022 9:24AM,247235,10,Emerginnova,,1
4,Sep 26 2022 9:22AM,247234,12,Hush Hush,,50
5,Sep 26 2022 9:20AM,247233,22,"NBTY Global, Inc.",1,
6,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation,,43
7,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation,,60
8,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation,,28
9,Sep 26 2022 8:46AM,247223,10,Bio-PRF,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 9:37AM,247239,10,Bio-PRF,1,
1,Sep 26 2022 9:36AM,247238,16,American International Chemical,1,
2,Sep 26 2022 9:28AM,247236,10,"Methapharm, Inc.",1,
3,Sep 26 2022 9:24AM,247235,10,Emerginnova,1,
4,Sep 26 2022 9:22AM,247234,12,Hush Hush,50,
5,Sep 26 2022 9:20AM,247233,22,"NBTY Global, Inc.",,1
6,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation,43,
7,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation,60,
8,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation,28,
9,Sep 26 2022 8:46AM,247223,10,Bio-PRF,9,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 9:37AM,247239,10,Bio-PRF,1,
1,Sep 26 2022 9:36AM,247238,16,American International Chemical,1,
2,Sep 26 2022 9:28AM,247236,10,"Methapharm, Inc.",1,
3,Sep 26 2022 9:24AM,247235,10,Emerginnova,1,
4,Sep 26 2022 9:22AM,247234,12,Hush Hush,50,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 9:37AM,247239,10,Bio-PRF,1.0,NaN
1,Sep 26 2022 9:36AM,247238,16,American International Chemical,1.0,NaN
2,Sep 26 2022 9:28AM,247236,10,"Methapharm, Inc.",1.0,NaN
3,Sep 26 2022 9:24AM,247235,10,Emerginnova,1.0,NaN
4,Sep 26 2022 9:22AM,247234,12,Hush Hush,50.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 9:37AM,247239,10,Bio-PRF,1.0,0.0
1,Sep 26 2022 9:36AM,247238,16,American International Chemical,1.0,0.0
2,Sep 26 2022 9:28AM,247236,10,"Methapharm, Inc.",1.0,0.0
3,Sep 26 2022 9:24AM,247235,10,Emerginnova,1.0,0.0
4,Sep 26 2022 9:22AM,247234,12,Hush Hush,50.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2719475,163.0,0.0
12,247234,50.0,0.0
16,247238,1.0,0.0
22,247233,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2719475,163.0,0.0
1,12,247234,50.0,0.0
2,16,247238,1.0,0.0
3,22,247233,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,163.0,0.0
1,12,50.0,0.0
2,16,1.0,0.0
3,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,163.0
1,12,Released,50.0
2,16,Released,1.0
3,22,Released,0.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,22
Status,,,,
Executing,0.0,0.0,0.0,1.0
Released,163.0,50.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,22
0,Executing,0.0,0.0,0.0,1.0
1,Released,163.0,50.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,22
0,Executing,0.0,0.0,0.0,1.0
1,Released,163.0,50.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()